<a href="https://colab.research.google.com/github/laurakrs/CPD2023-1/blob/main/CPD_Trabalho_Final_CERTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CLASSIFICACAO E PESQUISA DE DADOS - TRABALHO FINAL - FIFA21-PLAYERS

# ALUNAS:
# Laura Keidann Rodrigues da Silva (00217870)
# Raissa Bitencourt Borowski (00326579)

In [7]:
from collections import defaultdict
import csv
import time

In [ ]:
# Objetivo do trabalho: implementar estruturas de dados e algoritmos que suportam as seguintes pesquisas sobre os dados:

# - Pesquisas sobre os nomes de jogadores
# - Pesquisas sobre jogadores revisados por usuarios
# - Pesquisas sobre os melhores jogadores de uma determinada posicao
# - Pesquisas sobre ‘tags’ de jogadores

In [ ]:
# 1 - DADOS
# Os dados sao compostos dos arquivos players.csv, rating.csv e tags.csv, contendo
# respectivamente informaçoes sobre jogadores, avaliações de usuarios e anotacoes em
# texto-livre (tags).
# PLAYERS.CSV
# O arquivo players.csv contem informações de 18.944 jogadores, composto de um id no FIFA, seu nome e uma lista de posiçoes

In [ ]:
# RATING.CSV
# O arquivo rating.csv contem 24,188,078 de avaliaçees (notas entre 1 e 5) de usuarios para
# jogadores. Esses dados foram gerados simulando avaliacoes de usuários para cada jogador
# disponivel no dataset

In [ ]:
# ARQUIVO DE TESTE PARA RATINGS - MINIRATING.CSV
# Tambem disponibilizamos um arquivo com 10,000 avaliacos ao inves das 20 milhoes para
# ajudar nos testes (minirating.csv - disponivel para download apenas no Moodle). A leitura
# dos dados a partir do CSV pode ser lenta, em especial para o arquivo rating.csv que possui mais de 400MB de dados.

In [ ]:
# TAG.CSV
# O arquivo tag.csv contem 364,950 anotacoes de texto livre (tags) (ex.: Brazil, FK Specialist,
# Speedster, Playmaker, Paris Saint-Germain) para 18,944 jogadores.

In [8]:
# Classe jogador - player - para juntar as informacoes das 3 tabelas na entidade de cada jogador
class Player:
    def __init__(self, name, sofifa_id, player_positions):
        self.name = name
        self.id = sofifa_id
        self.player_positions = player_positions.split(', ')  # Transforma a string em lista
        self.tags = []
        # relativa a rating:
        self.count = 0
        self.sum = 0
        self.globalRating = 0.0

    def add_rating(self, rating):
        self.count += 1
        self.sum += rating
        self.globalRating = self.sum / self.count

    def add_tag(self, tag):
        self.tags.append(tag)

In [9]:
# Classe usuario - user - para juntar as informacoes dos usuarios
class User:
    def __init__(self, id):
        self.id = id
        self.ratedPlayers = []
        self.ratings = []

    def add_playerRating(self, rating):
        self.ratings.append(rating)

    def add_ratedPlayer(self, a_ratedPlayer):
        self.ratedPlayers.append(a_ratedPlayer)

In [10]:
# FUNCAO PARA LER A TABELA DE JOGADORES E INSERIR AS INFORMACOES NA INSTANCIA JOGADOR,
# INSERIR JOGADOR NA TABELA HASH
# INSERIR JOGADOR NA ARVORE TRIE

def readPlayers_csv(file):
  global playersList, playersTable, tamanhoPlayersTable

  with open(file, newline='') as csvfile:
        reader = csv.reader(csvfile) # Le o CSV
        next(reader)  # Pular a primeira linha

        # percorre os dados
        for row in reader:
            sofifa_id = int(row[0]) # primeiro elemento da linha
            name = row[1] # segundo elemento da linha
            player_positions = row[2] # terceiro elemento da linha

            a_player = Player(name, sofifa_id, player_positions) # cria um jogador

            playersList.append(a_player) # adicona o jogador na lista de todos os jogadores

            # Insere jogador na tabela Hash
            insereHash(playersTable, a_player.id, a_player, tamanhoPlayersTable)

            # Insere jogador na arvore Trie
            insertTrie(name, a_player)

In [11]:
# FUNCAO PARA LER A TABELA DE RATINGS E INSERIR OS VALORES NAS ENTIDADES APROPRIADAS

def readRatings_csv(file):
  global lastSofifaId, lastPlayer, playersTable, usersTable, usersList, tamanhoPlayersTable, tamanhoUsersTable

  with open(file, newline='') as csvfile:
        reader = csv.reader(csvfile) # Le o CSV
        next(reader)  # Pular a primeira linha

        # percorre os dados
        for row in reader:
          if len(row) == 3:
            user_id = int(row[0]) # primeiro elemento da linha
            sofifa_id = int(row[1]) # segundo elemento da linha
            rating = float(row[2]) # terceiro elemento da linha

            # confere valores recebidos
            if sofifa_id == lastSofifaId:
              player = lastPlayer
            else:
            # procura o jogador
              a_player = buscaHash(playersTable, sofifa_id, tamanhoPlayersTable)
              #if a_player is None: # ISSO AQUI FICOU ESTRANHO, ele tava procunrando umas ids que nao existiam
                #print(sofifa_id)
              #else:
              lastSofifaID = a_player.id
              lastPlayer = a_player

              # Adiciona o rating
              a_player.add_rating(rating)


            # Processa dados referentes ao usuario
            if user_id in usersList:
              # procura na tabela hash de usuarios
              user = buscaHash(usersTable, user_id, tamanhoUsersTable)
              # de qualquer forma, adiciona as avaliacoes no user
              user.add_ratedPlayer(a_player) #adiciona o rated player
              user.add_playerRating(rating) #adiciona o rating
            else: # se o usuario nao consta na lista ainda
              usersList.add(user_id) # adiciona no set
              # cria nova instancia
              user = User(user_id)
              # de qualquer forma, adiciona as avaliacoes no user
              user.add_ratedPlayer(a_player) #adiciona o rated player
              user.add_playerRating(rating) #adiciona o rating
              # adiciona na Hash Table de usuarios
              insereHash(usersTable,user_id,user,tamanhoUsersTable)




In [12]:
# FUNCAO PARA LER AS TAGS

def readTags_csv(file):
  global lastSofifaId, lastPlayer, playersTable, tamanhoPlayersTable

  with open(file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Pular a primeira linha

        for row in reader:
            sofifa_id = row[1]
            tag = row[2]

            # confere valores recebidos
            if sofifa_id == lastSofifaId:
              player = lastPlayer
            else:
              player = buscaHash(playersTable, int(sofifa_id), tamanhoPlayersTable)
              lastSofifaId = sofifa_id
              lastPlayer = player

            player.add_tag(tag) # adiciona a tag



In [ ]:
# TABELA HASH - IMPLEMENTACAO
# O tamanho da tabela M e variavel e deve ser informado na construcao
# A tabela hash deve armazenar registros compostos de uma chave identificadora e dados satelites associados
# Tarefas para implementacao:
# Escolha uma funcao de hash para mapear a chave de cada registro para um inteiro entre 0 e M −1;
# Implemente o metodo de resolucao de conflitos de enderecamento fechado com listas encadeadas;
# Implemente a operacao de insercao de um elemento na tabela hash;
# Implemente a operacao de busca de um nome na tabela hash

In [13]:
def funcaoPolinomial(id, tamanho):
    p = 53
    m = tamanho
    power_of_p = 1
    hash_key = 0

    # COMENTEI PARA TESTAR FUNCAO MAIS SIMPLES:
    #while id > 0:
    #     hash_key = ((hash_key + (id % 10) * power_of_p) % m)
    #     power_of_p = (power_of_p * p) % m
    #     id //= 10

    hash_key = id % tamanho # TESTAR SO FAZENDO O RESTO

    return int(hash_key)

In [14]:
# Operacao de Insercao na Tabela Hash(com listas encadeadas)

def insereHash(tabela, chave, valor, tamanho):
  # calcula a funcao hash
  hash_key = funcaoPolinomial(chave, tamanho)

  if tabela[hash_key] is None:
      tabela[hash_key] = []

  # adiciona elemento
  tabela[hash_key].append(valor)



In [15]:
# Operacao de Busca
# A funcao de busca recebe uma tabela hash, uma chave para buscar e o tamanho da tabela hash
# A funcao procura o elemento com a chave na tabela hash
# Se o elemento for encontrado, retorna o elemento
# Caso contrario, retorna -1

def buscaHash(tabela, chave, size):
    # Calcula hash
    i = funcaoPolinomial(chave, size)

    # Acha a posicao correta
    while tabela[i] is not None:
      # percorre os itens daquela posicao da tabela
      for item in tabela[i]:
        # se encontrar o item com a chave correta
        if item.id == chave:
          # retorna o resultado
          resultado = item
          return resultado
      # retorna -1 se foi um miss (o elemento nao foi encontrado na lista da sua posicao na tabela hash)
      #resultado -1
      #return resultado

    # retorna -1 se foi um miss (posicao vazia)
    #resultado = -1
    #return resultado

In [16]:
# ARVORE TRIE - IMPLEMENTACAO (inspirada no exemplo de implementacao em C++ disponivel no Moodle da disciplina)

# definicao da class TrieNode
class TrieNode:
    def __init__(self, char):
        self.char = char
        self.children = [None] * 26
        self.isEndOfWord = False
        self.player = None

# Retorna novo trie node
def get_node(char=None):
    return TrieNode(char)

# Funcao para inserir nova chave na arvore Trie
def insertTrie(key, instance):
    global root

    pCrawl = root # acessa raiz

    for char in key:
        charInChildren = False # premissa = nao encontrou char

        for child in pCrawl.children: # percorre os filhos

          if child != None and child.char == char: # verifica se tem um igual
            pCrawl = child # seta pCrawl como a child
            charInChildren = True # indica que encontrou
            break # para de percorrer

        if charInChildren == False: # se nao encontrou
          newNode = TrieNode(char) # novo no para a letra
          pCrawl.children.append(newNode)
          pCrawl = newNode # seta pCrawl como novo no

    pCrawl.isEndOfWord = True # seta como fim de palavra
    pCrawl.player = instance # coloca o jogador no no


# Funcao para buscar chave na arvore Trie # NAO USAR
def searchTrie(root, key):
    pCrawl = root
    for char in key:
        index = ord(char) - ord('a')
        if not pCrawl.children[index]:
            return False
        pCrawl = pCrawl.children[index]
    return pCrawl is not None and pCrawl.isEndOfWord


# FUNCAO PARA PROCURAR O PREFIXO NA TRIE

def searchPrefix(prefix):
  global root

  pCrawl = root # comeca pela raiz

  if len(root.children) == 0: # nao tem filhos
    print('None found')
    return None

  # se tem filhos
  for char in prefix:
    found = False

    for child in pCrawl.children:
      if child is not None:
        if child.char == char:
            found = True
            pCrawl = child
            break
    if found == False:
      print('None found')
      return None

  return pCrawl



In [17]:
# FUNCAO DE BUSCA GERAL

def searchInTrie(name):
  playersFound = findPlayers(name)

  if playersFound == None:
    print('None found!!!!')
    return
  else:
    printPlayers(playersFound)

In [18]:
# FUNCAO PARA JUNTAR TODOS OS JOGADORES ENCONTRADOS COM NOME/PREFIXO

def findPlayers(name):
  # lista vazia
  playersList = []

  lastNode = searchPrefix(name)

  if lastNode == None:
    return None
  else:
    completeList = searchTrieRec(lastNode, playersList) # chama busca recursiva
    return completeList



In [19]:
# FUNCAO DE BUSCA RECURSIVA

def searchTrieRec(node,playersList):
  if node is not None:
    if node.isEndOfWord == True:
      playersList.append(node.player) # junta o jogador
    else:
      for child in node.children:
        searchTrieRec(child,playersList)
    return playersList # retorna a lista


In [20]:
# PARTICIONAMENTO 1
# PARTICIONAMENTO LOMUTO

def particionamentoLomuto(lista, esq, dir):

    pivo = lista[esq]
    indice = esq + 1 #indice do menor

    i = esq + 1
    while (i <= dir):
        # se o elemento atual for menor ou igual ao pivo
          if lista[i][1] > pivo[1]:
            lista[i], lista[indice] = lista[indice], lista[i]
            indice += 1
          i += 1

    # troca de posicao
    lista[esq], lista[indice-1] = lista[indice-1], lista[esq]

    return (indice-1)




In [21]:
# QUICKSORT COM AS LISTAS
def quicksort(instancesList, valuesList):

    listToSort = []

    i = 0
    for player in instancesList:
        listToSort.append([player, valuesList[i]])
        i += 1

    quicksortRecursiva(listToSort, 0, (len(listToSort) - 1))

    sortedInstances = []
    sortedValues= []

    for pair in listToSort:
        sortedInstances.append(pair[0])
        sortedValues.append(pair[1])

    return sortedInstances, sortedValues

# Recursao do quicksort
def quicksortRecursiva(listToSort, esq, dir):
    if dir > esq:
        indicePivo = particionamentoLomuto(listToSort, esq, dir)
        quicksortRecursiva(listToSort, esq, indicePivo - 1)
        quicksortRecursiva(listToSort, indicePivo + 1, dir)


In [65]:
# FUNCAO PARA IMPRIMIR INFORMACOES DOS JOGADORES

def printPlayers(listOfPlayers):
  # Print header
  print("{:<15} {:<50} {:<25} {:<15} {:<15}".format("SOFIFA_ID", "NAME", "POSITIONS", "RATING", "COUNT"))
  print("="*120)
  for player in listOfPlayers:
      print("{:<15} {:<50} {:<25} {:<15.6f} {:<15}".format(str(player.id), player.name, str(player.player_positions), (player.globalRating), str(player.count)))

In [ ]:
# ARQUIVOS DE TESTE - Criacao da tabela Hash a partir dos dados dos jogadores - retirado do Lab4 - Tabelas Hash
# A implementacao da tabela Hash deve ser testada com um arquivo .CSV contendo descricoes de 18944 nomes de
# jogadores de futebol. O arquivo e disponibilizado no Moodle com o nome de players-fifa.csv. A primeira linha do
# arquivo contem a descricao das colunas, seguidos dos dados dos jogadores, um por linha
# Cada jogador e identificado por uma chave inteira (campo sofifa.id), seguido do nome e de uma lista de
# posicoes que o jogador atua

In [ ]:
# Arquivo minirating.csv om 10,000 avaliações ao invés das 20 milhões para ajudar nos testes
# minirating = readRatings_csv('/content/minirating.csv', 'rating')
#print(minirating)

In [ ]:
# TABELA HASH PARA AS AVALIACOES DE CADA JOGADOR:
# Guardar em uma TABELA HASH as medias de avaliacoes e o total de avaliacoes para cada jogador.

In [ ]:
# 2. PESQUISAS

In [ ]:
# 2.1 Pesquisas sobre os nomes de jogadores
# Esta pesquisa tem por objetivo retornar a lista de jogadores cujo nome começa com um texto
# que pode ser o prefixo ou texto completo do nome de um jogador. Todos os jogadores que
# satisfizerem o texto de consulta devem ser retornados, junto com a lista de posicoes dos
# jogadores, avaliacao media e numero de avaliacoes. A sintaxe dessa consulta eh:
# A consulta pode ser feita diretamente pelo console, e o resultado também pode ser impresso
# no console. Tente deixar o texto a ser impresso compacto.

# Exemplo: $ player Fer
# Retornar: sofifa_id, name, player_positions, rating, count dos jogadores cujos nomes se enquadram na pesquisa

In [ ]:
# exemplo:
# searchInTrie('Lionel')

In [ ]:
# Os dados de jogadores nao contem algumas das informacoes a serem retornadas. Por exemplo, deve-se primeiramente guardar em uma TABELA HASH as medias de avaliacoes
# e total de avaliacoes para cada jogador. Essas informacoes devem ser calculadas e armazenadas em uma etapa de pre-processamento.
# Para responder esta pesquisa, deve-se implementar uma ARVORE TRIE que busca todos os identificadores de
# jogadores que correspondem ao nome ou prefixo dado, e com essa lista de identificadores,
# buscar na TABELA HASH as informacoes complementares dos jogadores

In [ ]:
# TABELA HASH COM A MEDIA DE AVALIACOES E O TOTAL DE AVALIACOES PARA CADA JOGAR

In [ ]:
# ARVORE TRIE COM OS NOMES DOS JOGADORES PARA BUSCAR TODOS OS IDENTIFICADORES DE JOGADORES QUE  CORRESPONDEM AO NOME OU PREFIXO DADO

In [ ]:
# BUSCAR NA TABELA HASH AS INFORMACOES COMPLEMENTARES DOS JOGADORES

In [ ]:
# 2.2 Pesquisas sobre jogadores revisados por usuarios
# Esta pesquisa deve retornar a lista com no maximo 20 jogadores revisados pelo usuario e para
# cada jogador mostrar a nota dada pelo usuario, a media global e a contagem de avaliacoes. A
# lista deve ser ordenada pela nota dada pelo usuario de maneira decrescente (maiores notas
# primeiro). A sintaxe dessa consulta eh:
# user <userID>

# Exemplo: $ user 4
# Retornar: sofifa_id, name, global_rating, count, rating dos jogadores que o usuario revisou

# Os dados de entrada nao possuem diretamente as informacoes de usuarios
# prontas para o retorno dessa pesquisa. Estas informacoes estao colocadas no arquivo
# rating.csv. Deve-se implementar uma estrutura de dados (livre) para calcular e agregar em
# pre-processamento as informações por usuario. Com essa estrutura construida, para responder
# essa pesquisa basta consultar as entradas pelo identificador do usuario e retornar a lista de
# jogadores que esse usuario revisou e suas informacoes adicionais.

In [23]:
# FUNCAO PARA PROCURAR AS REVIEWS DE UM USUARIO
# A funcao recebe um usuario
def searchUserReviews(user_id):
  global usersTable, tamanhoPlayersTable

  selectedUser = buscaHash(usersTable, int(user_id), tamanhoUsersTable)

  if selectedUser == None:  # usuario nao encontrado
    return None
  else:
    # seleciona as informacoes do usuario
    playersReviewed = selectedUser.ratedPlayers
    playersRatings = selectedUser.ratings

    sortedPlayersReviewed, sortedPlayersRatings = quicksort(playersReviewed,playersRatings)

    top20playersReviewed = []

    i = 0
    while(len(top20playersReviewed) < 20 and i < len(sortedPlayersReviewed)):
      top20playersReviewed.append([sortedPlayersReviewed[i], sortedPlayersRatings[i]])
      i += 1

    return top20playersReviewed


In [24]:
# Funcao geral para pegar os dados encontrados das avaliacoes do usuario

def getUserReviews(user):
  listOfTopPlayers = searchUserReviews(user)

  if listOfTopPlayers == None:
    return print ('Nao tem avaliacoes')
  else:
    printPlayersReviewed(listOfTopPlayers)

In [71]:
# FUNCAO PARA IMPRIMIR OS TOP 20 JOGADORES AVALIADOS POR UM USUARIO
def printPlayersReviewed(listOfPlayersReviewed):
    # Print header
    print("{:<15} {:<50} {:<25} {:<15} {:<15}".format("SOFIFA_ID", "NAME", " GLOBAL RATING", "COUNT", "USER RATING"))
    print("="*120)
    for topPlayer, topRating in listOfPlayersReviewed:
      print("{:<15} {:<50} {:<25.6f} {:<15} {:<15.1f}".format(str(topPlayer.id), topPlayer.name, topPlayer.globalRating, str(topPlayer.count), topRating))

In [ ]:
# PESQUISAR REVIEWS

In [ ]:
# 2.3 Pesquisas sobre os melhores jogadores de uma determinada posicao
# Retornar a lista de jogadores com melhores notas de uma dada posicao
# Para evitar que um jogador seja retornando com uma boa media mas com poucas
# avaliacoes, esta consulta somente deve retornar os melhores jogadores com NO MINIMO 1000
# avaliacoes. Para gerenciar o numero de jogadores a serem retornados, a consulta deve
# receber como parametro um numero N que corresponde ao numero maximo de jogadores a
# serem retornados. O resultado da consulta deve estar ordenado da maior para a menor
# avaliaçao.
# Sintaxe da consulta: top<N> ‘<position>’
# Exemplo: $ top10 'ST''
# Retornar: sofifa_id, name, player_position, rating, count

In [58]:
def searchTopPlayersByPosition(position, max_results):
    global playersTable, tamanhoPlayersTable
    ## OU PEGA A LISTA DE INSTANCIAS DE JOGADORES
    global playersList

    allTopPlayers = []
    topPlayersRatings = []

    # Coletar todos os jogadores da posição especificada
    #for i in range(tamanhoPlayersTable):
    # ou
    for player in playersList: # vai direto nos jogadores
        # if playersTable[i] is not None:
            # for player in playersTable[i]:
                if position in player.player_positions:
                    if player.count >= 1000:
                        # Coloca o jogador na lista de jogadores
                        allTopPlayers.append(player)
                        # Coloca o global rating do jogador na lista de Ratings # FALTAVA ISSO AQUI
                        topPlayersRatings.append(player.globalRating)

    # Ordenar os jogadores pelo rating global em ordem decrescente
    sortedPlayers, sortedRatings = quicksort(allTopPlayers,topPlayersRatings) #REVISAR

    # Limitar o número de resultados
    allTopPlayers = sortedPlayers[:int(max_results)]

    # print(allTopPlayers)
    # Imprimir os dados dos jogadores
    printPlayers(allTopPlayers)



In [ ]:
# Exemplo de uso
searchTopPlayersByPosition('ST',10)

In [ ]:
# 2.4 Pesquisas sobre 'tags' de jogadores
# Explorar a lista de tags adicionadas por cada usuario em cada revisao.
# Para uma lista de tags dada como entrada, a pesquisa deve retornar a lista de
# jogadores que estao associados a intersecao de um conjunto de tags.
# Sintaxe da consulta: tags <list of tags>
# Exemplo: $ tags 'Brazil''Dribbler''
# Retornar: sofifa_id, name, player_position, rating, count, de todos os jogadores que apresentam tanto a tag '’Brazil" como
# a tag "Dribbler".

# Estas informações nao estao diretamente nos arquivos de dados. Portanto, deve-se projetar
# uma estrutura de dados para responder essa pesquisa. Como as tags podem ser termos com
# espaco (ex.: Solid Player, French Ligue 1, Manchester United), a tag passada na consulta deve
# ser escrita entre apostrofes.

In [43]:
# Funcao para procurar as tags nos jogadores e retornar a lista de jogadores com a tags indicadas pelo usuario

def searchTagsInPlayers(tagsEntrada):
  global playersList

  tagsEntradaSet = set(tagsEntrada)  # set das tags passadas por parametro
  playersTagged = [] # lista para os jogadores com as tags

  # percorre os jogadores da lista de jogadores
  for player in playersList:
    # se o set de tags passado pelo usuario pertence ao set de tags do jogador
    if tagsEntradaSet.issubset(player.tags):
      # adiciona o jogador na lista de jogadores com a tag desejada
      playersTagged.append(player)
  # Imprime a lista de jogadores com as tags
  printPlayers(playersTagged)




In [ ]:
# Exemplo:

# tags 'Brazil''Dribbler'
entrada = "tags 'Brazil''Dribbler'"
tags_user = entrada.split("'")[1::2] #separa os elementos
searchTagsInPlayers(tags_user)


In [ ]:
# 3 IMPLEMENTACAO
# Os usuarios devem construir uma aplicacao que funciona em duas fases.

# FASE 1 - CONSTRUCAO E INICIALIZACAO
# A fase 1 corresponde a construcao e inicializacao das estruturas de dados necessarias para suportar as consultas.
# Ao executar a fase de construcao, esta não deve demorar mais de 3 minutos. Quem conseguir
# fazer esta etapa em menos de 1 minuto ganha um bonus de 10%.

In [28]:
# MAIN - FASE 1 - CONSTRUCAO E INICIALIZACAO

tamanhoPlayersTable = 10000 # tamanho da tabela Hash de jogadores
tamanhoUsersTable = 20000

# INICIA O CRONOMETRO
startTime = time.perf_counter() # contador do tempo

# CRIACAO DA TABELA HASH PARA OS DADOS DOS JOGADORES
# cria a tabela vazia
playersTable = [None] * tamanhoPlayersTable
# Cria tabela de usuarios
usersTable = [None] * tamanhoUsersTable

# AROVRE TRIE
root = TrieNode('*')

# Inicializacao de variaveis
lastPlayer = None
lastSofifaId = ''


playersList = [] # todos os jogadores
topPlayers = [] # melhores jogadores com NO MINIMO 1000
topPlayersRatings = [] # avaliacoes dos melhores jogadores com NO MINIMO 1000
usersList = set() # usuarios

# CHAMADA DE FUNCOES PARA LER TABELAS E INSERIR NAS ESTRUTURAS

#  Chamadas de leituras para teste - pre-processamento
# Arquivo players.csv
# Le players.csv, cria instancia de Player, insere players na tabela hash e na arvore trie, adiciona player na lista
readPlayers_csv('/content/players.csv')

# Arquivo rating.csv
# Le rating.csv, vincula rating ao player em questao, atualiza ratings do player, cria instancia de user, adiciona user na Hash Table de users, adiciona user na lista de users
readRatings_csv('/content/rating.csv') # MAIS OU MENOS

# Arquivo tags.csv
# Le tags.csv, vincula tags ao jogador em questao
readTags_csv('/content/tags.csv')



# INTERROMPE O CRONOMETRO
endTime = time.perf_counter()
total_time = endTime - startTime

print(f'\nTempo de execucao da Fase 1 - Construcao e Inicializacao: {total_time:.2f} segundos')


Tempo de execucao da Fase 1 - Construcao e Inicializacao: 161.76 segundos


In [ ]:
# FASE 2 - MODO CONSOLE
# Depois dessas estruturas serem construidas, a aplicacao entra na fase 2, que corresponde ao
# modo console. Nesta fase sera possivel fazer as pesquisas listadas na secao anterior, bem
# como a exibicao dos resultados.

In [ ]:
# CHAMADAS DE TESTE DAS PESQUISAS


In [66]:
# 2.1
searchInTrie('Fer')
# searchInTrie('Lio')

SOFIFA_ID       NAME                                               POSITIONS                 RATING          COUNT          
135507          Fernando Luiz Rosa                                 ['CB', 'CDM']             3.720244        6402           
212618          Fernando Luis Aristeguieta de Luca                 ['ST']                    2.754755        999            
240482          Fernando Luis Bersano                              ['LB']                    2.664393        733            
202642          Fernando Lucas Martins                             ['CDM', 'CM']             2.981132        265            
178614          Fernando Lopes dos Santos Varela                   ['CB']                    2.793344        1292           
184134          Fernando Francisco Reges                           ['CDM', 'CM']             3.421439        4977           
228748          Fernando Fabian Fernandez Acosta                   ['ST']                    2.574462        1625           


In [72]:
# 2.2
getUserReviews(4)

SOFIFA_ID       NAME                                                GLOBAL RATING            COUNT           USER RATING    
176733          Marcus Berg                                        3.506993                  9367            5.0            
41              Andres Iniesta Lujan                               3.866787                  9680            5.0            
185221          Luiz Gustavo Dias                                  3.612475                  8633            5.0            
199206          Domagoj Vida                                       3.577764                  9098            5.0            
158963          Lucas Rodrigo Biglia                               3.529665                  6034            5.0            
168542          David Josue Jimenez Silva                          3.988069                  8717            4.5            
202053          R. Miyaichi                                        2.950679                  5525            4.5            


In [67]:
# 2.3
# top10 'ST''
searchTopPlayersByPosition('ST', 10)

SOFIFA_ID       NAME                                               POSITIONS                 RATING          COUNT          
158023          Lionel Andres Messi Cuccittini                     ['RW', 'ST', 'CF']        4.256382        12887          
20801           Cristiano Ronaldo dos Santos Aveiro                ['ST', 'LW']              4.247577        11144          
176580          Luis Alberto Suarez Diaz                           ['ST']                    4.174944        9403           
41236           Zlatan Ibrahimovic                                 ['ST']                    4.105496        9389           
188545          Robert Lewandowski                                 ['ST']                    4.081675        7989           
153079          Sergio Leonel Aguero del Castillo                  ['ST']                    4.052894        11249          
165153          Karim Benzema                                      ['CF', 'ST']              4.036720        10471          


In [68]:
# 2.4
# tags 'Brazil''Dribbler'
entrada = "tags 'Brazil''Dribbler'"
tags_user = entrada.split("'")[1::2] #separa os elementos
searchTagsInPlayers(tags_user)

SOFIFA_ID       NAME                                               POSITIONS                 RATING          COUNT          
190871          Neymar da Silva Santos Junior                      ['LW', 'CAM']             4.242804        17370          
201942          Roberto Firmino Barbosa de Oliveira                ['CF']                    3.802751        5488           
230658          Arthur Henrique Ramos de Oliveira Melo             ['CM']                    3.536732        2491           
190483          Douglas Costa de Souza                             ['LM', 'RW', 'LW']        3.704957        9663           
230666          Gabriel Fernando de Jesus                          ['ST']                    3.476959        4340           
176676          Marcelo Vieira da Silva Junior                     ['LB']                    3.929943        11562          
189242          Philippe Coutinho Correia                          ['CAM', 'LW', 'CM']       3.928441        7106           


In [ ]:
# MENU CONSOLE - INTERACAO COM O USUARIO

In [61]:
def menuConsole():
  print("\n\n" + "="*45 + " MENU: " + "="*45)
  print("\nSistema de Busca FIFA")
  print("\nDigite o comando da operacao desejada: ")
  print(  'player <name or prefix>    -> Lista de jogadores com o nome ou prefixo definidos \n'
          'user   <userID>            -> Lista com no máximo os 20 melhores jogadores revisados pelo usuário\n'
          'top<N> <position>          -> Lista de jogadores com melhores notas de uma dada posição\n'
          'tags   <list of tags>      -> Lista de jogadores com as tags\n')
  print("\nDigite 'SAIR' para sair")

  # LER ENTRADAS DO USUARIO

  entrada = input()

  # Identifica a entrada:
  if entrada.upper() == 'SAIR':
    print('Saindo...')
  elif entrada[:6] == 'player':
    searchInTrie(entrada[7:])  # Pesquisa o prefixo ou o nome na Trie dos jogadores e imprime a lista de jogadores com o nome ou prefixo definidos
    menuConsole()
  elif entrada[:4] == 'user':
    getUserReviews(entrada[5:])  # Pesquisa as reviews do usuario e imprime a lista com no máximo os 20 melhores jogadores revisados pelo usuário
    menuConsole()
  elif entrada[:3] == 'top': # Pesquisa e retorna a lista de jogadores com melhores notas da posicao
    entradaSeparada = entrada.split()  # Separa o input em 2 elementos
    searchTopPlayersByPosition(entradaSeparada[1][1:-1],int(entradaSeparada[0][3:]))
    menuConsole()
  elif entrada[:4] == 'tags':
    tagsSeparadas = entrada.split("'")[1::2]  # Separa as tags digitadas
    searchTagsInPlayers(tagsSeparadas)  # Pesquisa os jogadores com essas tags e imprime a lista de jogadores com as tags
    menuConsole()


  else:
    print('\nEntrada inválida. Tente novamente.\n')
    # Chama o menu de novo
    menuConsole()


In [62]:
menuConsole()



============================================= MENU: =============================================

Sistema de Busca FIFA

Digite o comando da operacao desejada: 
player <name or prefix>    -> Lista de jogadores com o nome ou prefixo definidos 
user   <userID>            -> Lista com no máximo os 20 melhores jogadores revisados pelo usuário
top<N> <position>          -> Lista de jogadores com melhores notas de uma dada posição
tags   <list of tags>      -> Lista de jogadores com as tags


Digite 'SAIR' para sair
top10 'ST'
SOFIFA_ID       NAME                                               POSITIONS                 RATING          COUNT          
158023          Lionel Andres Messi Cuccittini                     ['RW', 'ST', 'CF']        4.256382400869093 12887          
20801           Cristiano Ronaldo dos Santos Aveiro                ['ST', 'LW']              4.247577171572146 11144          
176580          Luis Alberto Suarez Diaz                           ['ST']                  